In [1]:
# Install packages
!pip install google-play-scraper pandas tqdm

In [3]:
# Import packages
from google_play_scraper import reviews, Sort
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import time

In [16]:
# Create a dictionary with google playstore ID's
apps = {
    'nike':'com.nike.omega',
    'adidas':'com.adidas.app',
    'puma':'com.puma.ecom.app',
    'gymshark':'com.gymshark.store'
}

In [17]:
# Create function to fetch reviews
def fetch_reviews(app_id, brand, lang='en', country='us', max_reviews = 1000, sleep = 3):
    all_rows = []
    token = None

    while len(all_rows) < max_reviews:
        batch, token = reviews(
            app_id,
            lang = lang,
            country = country,
            sort = Sort.NEWEST,
            count = min(200, max_reviews - len(all_rows)),
            continuation_token = token
        )

        for r in batch:
            all_rows.append({
                'brand':brand,
                'review_id':r.get('reviewID'),
                'score':r.get('score'),
                'at':r.get('at'),
                'content':r.get('content'),
                'reply_content':r.get('replyContent'),
                'thumbs_up':r.get('thumbsUpCount'),
                'review_created_version':r.get('reviewCreatedVersion')
            })

        if token is None:
            break
        time.sleep(sleep)

    return pd.DataFrame(all_rows)

In [ ]:
frames = []
for brand, pkg in apps.items():
    print(f'Fetching: {brand} ({pkg})')
    df = fetch_reviews(pkg, brand, max_reviews = 1000)
    frames.append(df)

reviews_df = pd.concat(frames, ignore_index=True)

Fetching: nike (com.nike.omega)
Fetching: adidas (com.adidas.app)
Fetching: puma (com.puma.ecom.app)
Fetching: gymshark (com.gymshark.store)
